# Toronto Neighborhood Analysis

In [242]:
from bs4 import BeautifulSoup
# !pip install geocoder
import geocoder

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1. Web Scriping - Toronto Neighborhood Data

In [2]:
# Using BeautifulSoup for web scraping
result = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
src = result.content
soup = BeautifulSoup(src,'lxml')

In [3]:
table = soup.find('table')
df = pd.read_html(str(table))[0]
df.head()

Postal Code           Borough               Neighborhood
0         M1A      Not assigned                        NaN
1         M2A      Not assigned                        NaN
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

In [4]:
# Pandas can do the same job on getting the table here
page = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df_nb = pd.DataFrame(page[0])
df_nb.head()

Postal Code           Borough               Neighborhood
0         M1A      Not assigned                        NaN
1         M2A      Not assigned                        NaN
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

### 2. Data Wrangling

In [5]:
# Dropping rwos on when borough is not assigned
df.drop(df[df["Borough"]=="Not assigned"].index,inplace = True)
df.reset_index(drop = True,inplace = True)
df.head()

Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [6]:
# check if "Not assigned" neighborhood
"Not assigned" in df["Neighborhood"].tolist()

False

In [7]:
# Showing shape of df
df.shape

(103, 3)

### 2. Adding Latitude and Longitude to Dataframe

##### 2.1 Geocoder Method (currenlty not working, thus, using csv)

In [8]:
# postal_code = 'M6S'
# lat_lng_coords = None
# geolocator = Nominatim(user_agent="Toronto_locator")

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#     g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#     lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

#### 2.2 Load Geo Info from CSV

In [9]:
df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [10]:
neighborhoods = df.merge(df_geo,on = 'Postal Code')
neighborhoods.head()

Postal Code           Borough                                 Neighborhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

In [11]:
# Check if any missing value
True in neighborhoods['Latitude'].isnull().tolist()

False

In [12]:
# Show # of boroughs and # of neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


### 3. Use Geopy to get coordination of Toronto

In [13]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


### 4. Create a map of Toronto with neighborhoods superimposed on top

In [14]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 5. Use Foursquare API to explore Toronto

##### 5.1 Set up Foursquare API parameters

In [3]:
# Create vareables to fit values in the folloing link:
# 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)

CLIENT_ID = 'KSH5WQJA5OQ0QTENV4NMZLREYMTPMFK1WIJO2Y24RSYS3LTA' # your Foursquare ID
CLIENT_SECRET = 'PDHUBJW1PPSFCVKL5ZPRLNV5SX5BXXT22TZOD2MPRR4PHQA3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # number of venues returned
radius = 500 # meters

# We also need latitude and longitude to complete url

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KSH5WQJA5OQ0QTENV4NMZLREYMTPMFK1WIJO2Y24RSYS3LTA
CLIENT_SECRET:PDHUBJW1PPSFCVKL5ZPRLNV5SX5BXXT22TZOD2MPRR4PHQA3


##### 5.2 Exploring Parkwoods Neighborhood and Analysing JSON Structure

In [42]:
nb_latitude = 43.753259
nb_longitude = -79.329656

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, nb_latitude, nb_longitude, VERSION, radius, LIMIT)

In [43]:
results = requests.get(url).json()

In [47]:
# Note for finding JSON structure

v_name = results['response']['groups'][0]['items'][0]['venue']['name']
v_name
# v_lat = results['response']['groups'][0]['items'][0]['venue']['location']['lat']
# v_lng = results['response']['groups'][0]['items'][0]['venue']['location']['lng']
# v_cat = results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']

'Brookbanks Park'

In [19]:
venues = results['response']['groups'][0]['items']
venues_nb = json_normalize(venues)
column_filters =  ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues_nb  = venues_nb.loc[:,column_filters]

In [20]:
for i in range(venues_nb.shape[0]):
    venues_nb.loc[i,'venue.categories'] = results['response']['groups'][0]['items'][i]['venue']['categories'][0]['name']

In [21]:
venues_nb.columns = [col.split('.')[-1] for col in venues_nb.columns.values]

In [59]:
venues_nb.head()

name         categories        lat        lng
0             Brookbanks Park               Park  43.751976 -79.332140
1               Variety Store  Food & Drink Shop  43.751974 -79.333114
2  TTC stop - 44 Valley Woods           Bus Stop  43.755402 -79.333741
3               GreenWin pool               Pool  43.756232 -79.333842

##### 5.3 Gather Venue Information for All Neighborhoods

In [81]:
# Define a function to fetch all venue data

def getNearbyVenues(hood_names, lats, lngs, radius=500):

    column_names = ['Neighborhood', 
                'Neighborhood Latitude', 
                'Neighborhood Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category']
    venues_nb = pd.DataFrame(columns=column_names)

    
    for i,name in enumerate(hood_names):
        print(name)
        nb_name  = name
        nb_latitude = lats[i]
        nb_longitude = lngs[i]
        
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, nb_latitude, nb_longitude, VERSION, radius, LIMIT)
        results = requests.get(url).json()
        venues = results['response']['groups'][0]['items']
        
        
        for i in range(len(venues)):
            v_name = results['response']['groups'][0]['items'][i]['venue']['name']
            v_lat = results['response']['groups'][0]['items'][i]['venue']['location']['lat']
            v_lng = results['response']['groups'][0]['items'][i]['venue']['location']['lng']
            v_cat = results['response']['groups'][0]['items'][i]['venue']['categories'][0]['name']
            
            # Note: DataFrame.append is not an in-place operation.
            venues_nb = venues_nb.append({'Neighborhood':nb_name,
                              'Neighborhood Latitude':nb_latitude,
                              'Neighborhood Longitude':nb_longitude,
                              'Venue':v_name,
                              'Venue Latitude':v_lat,
                              'Venue Longitude':v_lng,
                              'Venue Category':v_cat}, ignore_index=True)
            
    print('\n'+"Shape:",venues_nb.shape)
    return venues_nb

In [83]:
trt_venues = getNearbyVenues(hood_names = neighborhoods['Neighborhood'],
                                   lats=neighborhoods['Latitude'],
                                   lngs=neighborhoods['Longitude']
                                  )
trt_venues.head()

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2         Parkwoods              43.753259              -79.329656   
3         Parkwoods              43.753259              -79.329656   
4  Victoria Village              43.725882              -79.315572   

                        Venue  Venue Latitude  Venue Longitude  \
0             Brookbanks Park       43.751976       -79.332140   
1               Variety Store       43.751974       -79.333114   
2  TTC stop - 44 Valley Woods       43.755402       -79.333741   
3               GreenWin pool       43.756232       -79.333842   
4      Victoria Village Arena       43.723481       -79.315635   

      Venue Category  
0               Park  
1  Food & Drink Shop  
2           Bus Stop  
3               Pool  
4       Hockey Arena

In [129]:
# Check if any vanue type named as "Neighborhood"
print('Neighborhood' in trt_venues['Venue Category'].tolist())
trt_venues[trt_venues['Venue Category']=='Neighborhood'].head()

False


Empty DataFrame
Columns: [Neighborhood, Neighborhood Latitude, Neighborhood Longitude, Venue, Venue Latitude, Venue Longitude, Venue Category]
Index: []

In [130]:
# Drop row for vanue type named as "Neighborhood"
trt_venues = trt_venues.drop(trt_venues[trt_venues['Venue Category']=='Neighborhood'].index)
trt_venues.shape

(2108, 7)

In [131]:
trt_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Agincourt                                                               4   
Alderwood, Long Branch                                                 10   
Bathurst Manor, Wilson Heights, Downsview North                        19   
Bayview Village                                                         4   
Bedford Park, Lawrence Manor East                                      23   
Berczy Park                                                            56   
Birch Cliff, Cliffside West                                             4   
Brockton, Parkdale Village, Exhibition Place                           23   
Business reply mail Processing Centre, South Ce...                     17   
CN Tower, King and Spadina, Railway Lands, Harb...                     15   
Caledonia-Fairbanks                                                     3   
Canada Post Gateway Processing Centre                                  12   
Cedarbrae                                                               9   
Central Bay Street                                                     62   
Christie                                                               17   
Church and Wellesley                                                   76   
Clarks Corners, Tam O'Shanter, Sullivan                                14   
Cliffside, Cliffcrest, Scarborough Village West                         2   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             34   
Davisville North                                                        7   
Del Ray, Mount Dennis, Keelsdale and Silverthorn                        6   
Don Mills                                                              27   
Dorset Park, Wexford Heights, Scarborough Town ...                      7   
Downsview                                                              16   
Dufferin, Dovercourt Village                                           14   
East Toronto, Broadview North (Old East York)                           3   
Eringate, Bloordale Gardens, Old Burnhamthorpe,...                      8   
Fairview, Henry Farm, Oriole                                           64   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         5   
Garden District, Ryerson                                              100   
Glencairn                                                               4   
Golden Mile, Clairlea, Oakridge                                        10   
Guildwood, Morningside, West Hill                                       7   
Harbourfront East, Union Station, Toronto Islands                      99   
High Park, The Junction South                                          23   
Hillcrest Village                                                       4   
Humber Summit                                                           1   
Humberlea, Emery                                                        1   
Humewood-Cedarvale                                                      3   
India Bazaar, The Beaches West                                         20   
Kennedy Park, Ionview, East Birchmount Park                             5   
Kensington Market, Chinatown, Grange Park                              58   
Kingsview Village, St. Phillips, Martin Grove G...                      4   
Lawrence Manor, Lawrence Heights                                       11   
Lawrence Park                                                           4   
Leaside                                                                33   
Little Portugal, Trinity                                               43   
Malvern, Rouge                                                          1   
Milliken, Agincourt North, Steeles East, L'Amor.

In [132]:
print('There are {} uniques categories.'.format(len(trt_venues['Venue Category'].unique())))

There are 271 uniques categories.


#### 6. Analyze Each Neighborhood for Frequent Venue Categories

Methodology:  
    1. Use .get_dummies() method to do one hot encoding.  
    2. Find frequency of each type of venue occured for each neighborhood.   
    3. Find the most frequent ones by sorting the frequency column on tranposed table.  
      
Create a table to show all neighborhoods and top 10 common venue categories in that neighborhood:  
    1. Create a empty dataframe with column names ['Neighborhoods','1st Most Common Venue Type','2nd Most...','3rd Most...',...].  
    2. Fill in 'Neighborhoods' columns from values in dataframe:manhattan_venues.  
    3. Fill the rest neighborhood by neighborhood. Sort vanues by frequency info in manhattan_venues table, and extract most common venue types from it. Fill the empty table using this extracted values.  

##### 6.1 One-hot encoding

In [138]:
# onehot encoding
trt_onehot = pd.get_dummies(trt_venues[['Venue Category']], prefix="", prefix_sep="")

# Add 'Neighborhood' column to the table
trt_onehot['Neighborhood'] = trt_venues['Neighborhood']

# moving 'Neighborhood' column to the front
columns = ['Neighborhood'] + trt_onehot.columns[:-1].tolist()
trt_onehot = trt_onehot[columns]

trt_onehot.head()

Neighborhood  Accessories Store  Afghan Restaurant  Airport  \
0         Parkwoods                  0                  0        0   
1         Parkwoods                  0                  0        0   
2         Parkwoods                  0                  0        0   
3         Parkwoods                  0                  0        0   
4  Victoria Village                  0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0                 0                    0             0         0            0   
1                 0                    0             0         0            0   
2                 0                    0             0         0            0   
3                 0                    0             0         0            0   
4                 0                    0             0         0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0           0                    0                 0                   0   
1           0                    0                 0                   0   
2           0                    0                 0                   0   
3           0                    0                 0                   0   
4           0                    0                 0                   0   

   Auto Garage  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  \
0            0              0          0           0           0       0   
1            0              0          0           0           0       0   
2            0              0          0           0           0       0   
3            0              0          0           0           0       0   
4            0              0          0           0           0       0   

   Bank  Bar  Baseball Field  Baseball Stadium  Basketball Stadium  Beach  \
0     0    0               0                 0                   0      0   
1     0    0               0                 0                   0      0   
2     0    0               0                 0                   0      0   
3     0    0               0                 0                   0      0   
4     0    0               0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bike Shop  \
0                0         0           0                   0          0   
1                0         0           0                   0          0   
2                0         0           0                   0          0   
3                0         0           0                   0          0   
4                0         0           0                   0          0   

   Bistro  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0       0              0          0         0                     0   
1       0              0          0         0                     0   
2       0              0          0         0                     0   
3       0              0          0         0                     0   
4       0              0          0         0                     0   

   Breakfast Spot  Brewery  Bridal Shop  Bubble Tea Shop  Building  \
0               0        0            0                0         0   
1               0        0            0                0         0   
2               0        0            0                0         0   
3               0        0            0                0         0   
4               0        0            0                0     

In [165]:
trt_onehot = trt_onehot.groupby('Neighborhood').mean().reset_index()
trt_onehot.head()

Neighborhood  Accessories Store  \
0                                        Agincourt                0.0   
1                           Alderwood, Long Branch                0.0   
2  Bathurst Manor, Wilson Heights, Downsview North                0.0   
3                                  Bayview Village                0.0   
4                Bedford Park, Lawrence Manor East                0.0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                0.0      0.0                 0.0           0.0   
1                0.0      0.0                 0.0           0.0   
2                0.0      0.0                 0.0           0.0   
3                0.0      0.0                 0.0           0.0   
4                0.0      0.0                 0.0           0.0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0             0.0              0.0               0.0             0.000000   
1             0.0              0.0               0.0             0.000000   
2             0.0              0.0               0.0             0.000000   
3             0.0              0.0               0.0             0.000000   
4             0.0              0.0               0.0             0.043478   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0           0.0       0.0          0.0         0.0                  0.0   
1           0.0       0.0          0.0         0.0                  0.0   
2           0.0       0.0          0.0         0.0                  0.0   
3           0.0       0.0          0.0         0.0                  0.0   
4           0.0       0.0          0.0         0.0                  0.0   

   Asian Restaurant  Athletics & Sports  Auto Garage  Auto Workshop  \
0               0.0                 0.0          0.0            0.0   
1               0.0                 0.1          0.0            0.0   
2               0.0                 0.0          0.0            0.0   
3               0.0                 0.0          0.0            0.0   
4               0.0                 0.0          0.0            0.0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery      Bank  Bar  Baseball Field  \
0        0.0         0.0         0.0     0.0  0.000000  0.0             0.0   
1        0.0         0.0         0.0     0.0  0.000000  0.0             0.0   
2        0.0         0.0         0.0     0.0  0.105263  0.0             0.0   
3        0.0         0.0         0.0     0.0  0.250000  0.0             0.0   
4        0.0         0.0         0.0     0.0  0.000000  0.0             0.0   

   Baseball Stadium  Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  \
0               0.0                 0.0    0.0              0.0       0.0   
1               0.0                 0.0    0.0              0.0       0.0   
2               0.0                 0.0    0.0              0.0       0.0   
3               0.0                 0.0    0.0              0.0       0.0   
4               0.0                 0.0    0.0              0.0       0.0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  \
0         0.0                 0.0        0.0     0.0            0.0   
1         0.0                 0.0        0.0     0.0            0.0   
2         0.0                 0.0        0.0     0.0            0.0   
3         0.0                 0.0        0.0     0.0            0.0   
4         0.0                 0.0        0.0     0.0            0.0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0        0.0       0.0                   0.0            0.25      0.0   
1        0.0       0.0                   0.0            0.00      0.0   
2        0.0       0.0                   0.0            0.00      0.0   
3        0.0       0.0                   0.0            0.00      0.0   
4        0.0       0.0                   0.0            0.00      0.0   

   Bridal Shop  Bubble Tea Shop  Building  Burger J

###### To Confirm the size of this one-hot encoded dataset

In [179]:
trt_onehot.shape

(95, 272)

#### 6.2 Let's print each neighborhood along with the top 5 most common venues

In [178]:
# Prepare to sort the dataframe for each neighborhood
trt_vrank = trt_onehot.iloc[:,1:trt_onehot.shape[1]].T
trt_vrank.columns = trt_onehot['Neighborhood']
trt_vrank.head()

Neighborhood        Agincourt  Alderwood, Long Branch  \
Accessories Store         0.0                     0.0   
Afghan Restaurant         0.0                     0.0   
Airport                   0.0                     0.0   
Airport Food Court        0.0                     0.0   
Airport Gate              0.0                     0.0   

Neighborhood        Bathurst Manor, Wilson Heights, Downsview North  \
Accessories Store                                               0.0   
Afghan Restaurant                                               0.0   
Airport                                                         0.0   
Airport Food Court                                              0.0   
Airport Gate                                                    0.0   

Neighborhood        Bayview Village  Bedford Park, Lawrence Manor East  \
Accessories Store               0.0                                0.0   
Afghan Restaurant               0.0                                0.0   
Airport                         0.0                                0.0   
Airport Food Court              0.0                                0.0   
Airport Gate                    0.0                                0.0   

Neighborhood        Berczy Park  Birch Cliff, Cliffside West  \
Accessories Store           0.0                          0.0   
Afghan Restaurant           0.0                          0.0   
Airport                     0.0                          0.0   
Airport Food Court          0.0                          0.0   
Airport Gate                0.0                          0.0   

Neighborhood        Brockton, Parkdale Village, Exhibition Place  \
Accessories Store                                            0.0   
Afghan Restaurant                                            0.0   
Airport                                                      0.0   
Airport Food Court                                           0.0   
Airport Gate                                                 0.0   

Neighborhood        Business reply mail Processing Centre, South Central Letter Processing Plant Toronto  \
Accessories Store                                                 0.0                                      
Afghan Restaurant                                                 0.0                                      
Airport                                                           0.0                                      
Airport Food Court                                                0.0                                      
Airport Gate                                                      0.0                                      

Neighborhood        CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport  \
Accessories Store                                            0.000000                                                            
Afghan Restaurant                                            0.000000                                                            
Airport                                                      0.066667                                                            
Airport Food Court                                           0.066667                                                            
Airport Gate                                                 0.066667                                                            

Neighborhood        Caledonia-Fairbanks  \
Accessories Store                   0.0   
Afghan Restaurant                   0.0   
Airport                             0.0   
Airport Food Court                  0.0   
Airport Gate                        0.0   

Neighborhood        Canada Post Gateway Processing Centre  Cedarbrae  \
Accessories Store                                     0.0        0.0   
Afghan Restaurant                                     0.0        0.0   
Airport                                               0.0        0.0   
Airport Food C

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
# Sort venue cat for each neighborhood

nbs = trt_vrank.columns.tolist()
top_vcat = []

for nb in nbs:
    temp = trt_vrank[nb].sort_values(ascending=False).reset_index(drop=False)
    temp.columns = ['Venues', 'Freq']
#     print("\n\n-----"+nb+"-----\n")
#     print(temp.head(5))
#     temp = list(temp.iloc[0:10].index.values)
    temp = temp.iloc[0:10]
    temp = temp["Venues"].tolist()
    top_vcat.append(temp)

#### Let's put that into a *pandas* dataframe

In [206]:
# Generating columns
num_top_venues = 10
columns = ['Neighborhood']
nbring = ['st', 'nd', 'rd']

for i in range(10):
    if i<3:
        columns.append('{}{} Most Common Venue'.format(i+1,nbring[i]))
    else:
        columns.append('{}th Most Common Venue'.format(i+1))
columns

['Neighborhood',
 '1st Most Common Venue',
 '2nd Most Common Venue',
 '3rd Most Common Venue',
 '4th Most Common Venue',
 '5th Most Common Venue',
 '6th Most Common Venue',
 '7th Most Common Venue',
 '8th Most Common Venue',
 '9th Most Common Venue',
 '10th Most Common Venue']

In [238]:
nb_vcat = pd.DataFrame(columns = columns)
nb_vcat['Neighborhood'] = trt_onehot['Neighborhood'] 

for i in range(nb_vcat.shape[0]):
    nb_vcat.iloc[i,1:] = top_vcat[i] # You may assign list of values to a row

print(nb_vcat.shape)
nb_vcat.head()

(95, 11)


Neighborhood 1st Most Common Venue  \
0                                        Agincourt                Lounge   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
3                                  Bayview Village                  Café   
4                Bedford Park, Lawrence Manor East    Italian Restaurant   

       2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0  Latin American Restaurant        Breakfast Spot          Skating Rink   
1                        Gym              Pharmacy    Athletics & Sports   
2                       Bank         Shopping Mall              Pharmacy   
3                       Bank    Chinese Restaurant   Japanese Restaurant   
4             Sandwich Place      Sushi Restaurant           Coffee Shop   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0           Yoga Studio            Donut Shop        Discount Store   
1                   Pub        Sandwich Place          Dance Studio   
2        Ice Cream Shop      Sushi Restaurant         Deli / Bodega   
3    Dim Sum Restaurant                 Diner        Discount Store   
4             Juice Bar            Restaurant           Pizza Place   

       8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0        Distribution Center               Dog Run       Doner Restaurant  
1                Coffee Shop          Skating Rink              Drugstore  
2  Middle Eastern Restaurant            Restaurant    Fried Chicken Joint  
3        Distribution Center               Dog Run            Yoga Studio  
4            Thai Restaurant              Pharmacy      Indian Restaurant

#### 7. Cluster Neighborhoods

In [235]:
# set number of clusters
kclusters = 5

trt_clustering = trt_onehot.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(trt_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       2, 1, 1, 1, 1, 1, 2])

##### Attached Neighborhood lable to nb_vcat and merge with neighborhoods dataframe

In [239]:
nb_vcat.insert(0,'Cluster Labels',kmeans.labels_)

neighborgoods_merged = neighborhoods.merge(nb_vcat, left_on = "Neighborhood", right_on = "Neighborhood", how = "inner")
neighborgoods_merged.head()

Postal Code           Borough                                 Neighborhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  Cluster Labels  1st Most Common Venue  \
0  43.753259 -79.329656               1                   Park   
1  43.725882 -79.315572               1  Portuguese Restaurant   
2  43.654260 -79.360636               1            Coffee Shop   
3  43.718518 -79.464763               1      Accessories Store   
4  43.662301 -79.389494               1            Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0              Bus Stop     Food & Drink Shop                  Pool   
1           Coffee Shop          Intersection          Hockey Arena   
2                Bakery                   Pub                  Park   
3           Coffee Shop           Event Space    Miscellaneous Shop   
4           Yoga Studio         Burrito Place                  Bank   

    5th Most Common Venue       6th Most Common Venue  \
0     Distribution Center                Dessert Shop   
1       French Restaurant  Financial or Legal Service   
2                 Theater              Breakfast Spot   
3  Furniture / Home Store                    Boutique   
4                     Bar                    Beer Bar   

         7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0           Dim Sum Restaurant                 Diner        Discount Store   
1  Eastern European Restaurant             Drugstore            Donut Shop   
2                         Café            Shoe Store            Restaurant   
3          Arts & Crafts Store        Clothing Store          Carpet Store   
4           Italian Restaurant         Smoothie Shop           Music Venue   

  10th Most Common Venue  
0                Dog Run  
1           Dessert Shop  
2         Chocolate Shop  
3          Women's Store  
4         Sandwich Place

##### 6.3 Plot neighborhood cluster on map

In [241]:
# Locate toronto
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [244]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborgoods_merged['Latitude'], neighborgoods_merged['Longitude'], neighborgoods_merged['Neighborhood'], neighborgoods_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 7. Examine Clusters

##### Cluster 1 - Golf Friendly

In [250]:
neighborgoods_merged.loc[neighborgoods_merged['Cluster Labels'] == 0, neighborgoods_merged.columns[[1,2] + list(range(5, neighborgoods_merged.shape[1]))]]

Borough                                       Neighborhood  \
11  Etobicoke  West Deane Park, Princess Gardens, Martin Grov...   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
11               0           Golf Course           Yoga Studio   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
11            Donut Shop    Dim Sum Restaurant                 Diner   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
11        Discount Store   Distribution Center               Dog Run   

   9th Most Common Venue 10th Most Common Venue  
11      Doner Restaurant              Drugstore

##### Cluster 2 - Typical Downtown - Commercial Friendly

In [251]:
neighborgoods_merged.loc[neighborgoods_merged['Cluster Labels'] == 1, neighborgoods_merged.columns[[1,2] + list(range(5, neighborgoods_merged.shape[1]))]]

Borough                                       Neighborhood  \
0         North York                                          Parkwoods   
1         North York                                   Victoria Village   
2   Downtown Toronto                          Regent Park, Harbourfront   
3         North York                   Lawrence Manor, Lawrence Heights   
4   Downtown Toronto        Queen's Park, Ontario Provincial Government   
6         North York                                          Don Mills   
7         North York                                          Don Mills   
8          East York                    Parkview Hill, Woodbine Gardens   
9   Downtown Toronto                           Garden District, Ryerson   
10        North York                                          Glencairn   
12       Scarborough             Rouge Hill, Port Union, Highland Creek   
13         East York                                   Woodbine Heights   
14  Downtown Toronto                                     St. James Town   
15              York                                 Humewood-Cedarvale   
16         Etobicoke  Eringate, Bloordale Gardens, Old Burnhamthorpe...   
17       Scarborough                  Guildwood, Morningside, West Hill   
18      East Toronto                                        The Beaches   
19  Downtown Toronto                                        Berczy Park   
20              York                                Caledonia-Fairbanks   
21       Scarborough                                             Woburn   
22         East York                                            Leaside   
23  Downtown Toronto                                 Central Bay Street   
24  Downtown Toronto                                           Christie   
25       Scarborough                                          Cedarbrae   
26        North York                                  Hillcrest Village   
27        North York    Bathurst Manor, Wilson Heights, Downsview North   
28         East York                                   Thorncliffe Park   
29  Downtown Toronto                           Richmond, Adelaide, King   
30      West Toronto                       Dufferin, Dovercourt Village   
31       Scarborough                                Scarborough Village   
32        North York                       Fairview, Henry Farm, Oriole   
33        North York                    Northwood Park, York University   
35  Downtown Toronto  Harbourfront East, Union Station, Toronto Islands   
36      West Toronto                           Little Portugal, Trinity   
37       Scarborough        Kennedy Park, Ionview, East Birchmount Park   
38        North York                                    Bayview Village   
39        North York                                          Downsview   
40        North York                                          Downsview   
41        North York                                          Downsview   
42        North York                                          Downsview   
43      East Toronto                       The Danforth West, Riverdale   
44  Downtown Toronto           Toronto Dominion Centre, Design Exchange   
45      West Toronto       Brockton, Parkdale Village, Exhibition Place   
46       Scarborough                    Golden Mile, Clairlea, Oakridge   
47      East Toronto                     India Bazaar, The Beaches West   
48  Downtown Toronto                     Commerce Court, Victoria Hotel   
49        North York           North Park, Maple Leaf Park, Upwood Park   
50        North York                                      Humber Summit   
51       Scarborough    Cliffside, Cliffcrest, Scarborough Village West   
52      East Toronto                                    Studio District   
53        North York                  Bedford Park, Lawrence Manor East   
54              York   Del Ray, Mount Dennis, Keelsdale and Silverthorn   
56       Scarborough                  

##### Cluster 3 - Park Lover's Neighborhoods

In [252]:
neighborgoods_merged.loc[neighborgoods_merged['Cluster Labels'] == 2, neighborgoods_merged.columns[[1,2] + list(range(5, neighborgoods_merged.shape[1]))]]

Borough                                       Neighborhood  \
34         East York      East Toronto, Broadview North (Old East York)   
61              York                                             Weston   
63        North York                                    York Mills West   
82       Scarborough  Milliken, Agincourt North, Steeles East, L'Amo...   
88  Downtown Toronto                                           Rosedale   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
34               2                  Park     Convenience Store   
61               2                  Park      Doner Restaurant   
63               2                  Park                  Bank   
82               2                  Park            Playground   
88               2                  Park                 Trail   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
34    Falafel Restaurant           Event Space  Ethiopian Restaurant   
61          Dessert Shop    Dim Sum Restaurant                 Diner   
63     Convenience Store            Donut Shop                 Diner   
82               Dog Run      Department Store          Dessert Shop   
88            Playground      Department Store          Dessert Shop   

   6th Most Common Venue        7th Most Common Venue 8th Most Common Venue  \
34     Electronics Store  Eastern European Restaurant             Drugstore   
61        Discount Store          Distribution Center               Dog Run   
63        Discount Store          Distribution Center               Dog Run   
82    Dim Sum Restaurant                        Diner        Discount Store   
88    Dim Sum Restaurant                        Diner        Discount Store   

   9th Most Common Venue 10th Most Common Venue  
34      Department Store             Donut Shop  
61            Donut Shop   Gym / Fitness Center  
63      Doner Restaurant              Drugstore  
82   Distribution Center            Yoga Studio  
88   Distribution Center                Dog Run

##### Cluster 4 - Fast Food Friendly

In [253]:
neighborgoods_merged.loc[neighborgoods_merged['Cluster Labels'] == 3, neighborgoods_merged.columns[[1,2] + list(range(5, neighborgoods_merged.shape[1]))]]

Borough    Neighborhood  Cluster Labels 1st Most Common Venue  \
5  Scarborough  Malvern, Rouge               3  Fast Food Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
5      Doner Restaurant          Dessert Shop    Dim Sum Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
5                 Diner        Discount Store   Distribution Center   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
5               Dog Run           Yoga Studio       Department Store

##### Cluster 5 - Baseball Friendly

In [254]:
neighborgoods_merged.loc[neighborgoods_merged['Cluster Labels'] == 4, neighborgoods_merged.columns[[1,2] + list(range(5, neighborgoods_merged.shape[1]))]]

Borough                                       Neighborhood  \
55  North York                                   Humberlea, Emery   
97   Etobicoke  Old Mill South, King's Mill Park, Sunnylea, Hu...   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
55               4        Baseball Field           Yoga Studio   
97               4             Locksmith        Baseball Field   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
55    Dim Sum Restaurant                 Diner        Discount Store   
97                 Diner        Discount Store   Distribution Center   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
55   Distribution Center               Dog Run      Doner Restaurant   
97               Dog Run      Doner Restaurant            Donut Shop   

   9th Most Common Venue 10th Most Common Venue  
55            Donut Shop   Fast Food Restaurant  
97             Drugstore   Fast Food Restaurant